In [1]:
import numpy as np
import os
import pandas as pd
import re
from plotfx import *
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter

%matplotlib inline

### Restructure Results: No NPI, All Betas, High Community Transmission

In [3]:
sims = ['beta003_noNPI_vax_hicom', 'beta006_noNPI_vax_hicom', 'beta009_noNPI_vax_hicom']
partitions = np.arange(5)
sim_names = [sim + '_detailed_' + str(part) + '.csv' for sim in sims for part in partitions]
outfile = 'allbeta_noNPI_vax_hicom'

In [4]:
sim_names

['beta003_noNPI_vax_hicom_detailed_0.csv',
 'beta003_noNPI_vax_hicom_detailed_1.csv',
 'beta003_noNPI_vax_hicom_detailed_2.csv',
 'beta003_noNPI_vax_hicom_detailed_3.csv',
 'beta003_noNPI_vax_hicom_detailed_4.csv',
 'beta006_noNPI_vax_hicom_detailed_0.csv',
 'beta006_noNPI_vax_hicom_detailed_1.csv',
 'beta006_noNPI_vax_hicom_detailed_2.csv',
 'beta006_noNPI_vax_hicom_detailed_3.csv',
 'beta006_noNPI_vax_hicom_detailed_4.csv',
 'beta009_noNPI_vax_hicom_detailed_0.csv',
 'beta009_noNPI_vax_hicom_detailed_1.csv',
 'beta009_noNPI_vax_hicom_detailed_2.csv',
 'beta009_noNPI_vax_hicom_detailed_3.csv',
 'beta009_noNPI_vax_hicom_detailed_4.csv']

In [5]:
df = pd.DataFrame()
for file in sim_names:
    tmpdf = pd.read_csv(file, header = 0, dtype = {'rep': int, 'tstep': int, 'p_vax': float, 
                                                                  'test_freq': int})
    part = int(file[-5])
    beta = int(file[6])
    tmpdf['partition'] = np.repeat(part, repeats = tmpdf.shape[0])
    tmpdf['beta'] = np.repeat(beta, repeats = tmpdf.shape[0])
    df = df.append(tmpdf, ignore_index = True)

In [6]:
# Linspace adds trailing decimal places in some cases
df['p_vax'] = df['p_vax'].round(decimals = 1)

In [7]:
df.shape

(847500, 13)

In [8]:
df.head()

,rep,p_vax,tstep,ext_inf_ct,test_freq,ia_nodes,is_nodes,test_pos_schd,test_pos_symp,q_schd,q_symp,partition,beta
0,1,0.0,0,128,0,[],[684],[],[],[],[],0,3
1,1,0.0,1,128,0,[],[684],[],[],[],[],0,3
2,1,0.0,2,128,0,"[690, 159]",[684],[],[],[],[],0,3
3,1,0.0,3,128,0,"[690, 159]","[684, 661]",[],[],[],[],0,3
4,1,0.0,4,128,0,[159],"[684, 661]",[],[684],[],[],0,3


In [9]:
# Renumber reps to remove duplicates
df['rep'] = df['rep'] + df['partition']*20
df.tail()

,rep,p_vax,tstep,ext_inf_ct,test_freq,ia_nodes,is_nodes,test_pos_schd,test_pos_symp,q_schd,q_symp,partition,beta
847495,100,0.8,108,47,28,"[346, 303]","[305, 425, 17, 651, 294, 607, 344, 462, 474, 4...","[179, 111]",[444],"[521, 609, 131, 177, 294, 300, 336, 346, 440, ...","[521, 336, 474, 607, 230]",4,9
847496,100,0.8,109,47,28,"[346, 303]","[305, 425, 17, 294, 607, 344, 462, 444, 656, 4...",[411],[656],"[521, 609, 111, 131, 177, 179, 300, 336, 346, ...","[521, 444, 336, 474]",4,9
847497,100,0.8,110,47,28,"[346, 303]","[305, 425, 17, 607, 344, 462, 444, 656, 488, 526]",[],[],"[521, 609, 111, 131, 177, 179, 300, 336, 346, ...","[521, 656, 444, 474, 230]",4,9
847498,100,0.8,111,47,28,"[346, 303, 128]","[305, 425, 17, 607, 344, 462, 444, 488, 526, 1...",[],[],"[111, 131, 177, 179, 300, 346, 411, 440, 441]","[521, 656, 444, 474]",4,9
847499,100,0.8,112,47,28,"[303, 128]","[305, 425, 17, 607, 344, 462, 444, 488, 526, 1...",[],[],"[111, 179, 300, 346, 411, 440, 441]","[521, 656, 444, 474, 230]",4,9


In [10]:
"""RESTRUCTURE DATA"""

df['ia_nodes'] = df['ia_nodes'].apply(eval)
df['is_nodes'] = df['is_nodes'].apply(eval)
df['all_inf'] = df['ia_nodes'] + df['is_nodes']
df['n_inf'] = df['all_inf'].apply(set).apply(len)
df['cum_list_inf'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_inf'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_inf'] = df['cum_list_inf'].apply(len)

df['test_pos_schd'] = df['test_pos_schd'].apply(eval)
df['test_pos_symp'] = df['test_pos_symp'].apply(eval)
df['all_test'] = df['test_pos_schd'] + df['test_pos_symp']
df['n_test_pos'] = df['all_test'].apply(set).apply(len)
df['cum_list_test'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_test'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_pos'] = df['cum_list_test'].apply(len)
        
df['q_schd'] = df['q_schd'].apply(eval)
df['q_symp'] = df['q_symp'].apply(eval)
df['all_quar'] = df['q_schd'] + df['q_symp']
df['n_quar'] = df['all_quar'].apply(set).apply(len)
df['cum_list_quar'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_quar'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_quar'] = df['cum_list_quar'].apply(len)

In [ ]:
# Save out reformatted dataframe
filehandler = open(outfile + ".pkl","wb")
pickle.dump(df,filehandler)
filehandler.close()

### Restructure Results: No NPI, All Betas, Low Community Transmission

In [11]:
sims = ['beta003_noNPI_vax_locom', 'beta006_noNPI_vax_locom', 'beta009_noNPI_vax_locom']
partitions = np.arange(5)
sim_names = [sim + '_detailed_' + str(part) + '.csv' for sim in sims for part in partitions]
outfile = 'allbeta_noNPI_vax_locom'

In [12]:
sim_names

['beta003_noNPI_vax_locom_detailed_0.csv',
 'beta003_noNPI_vax_locom_detailed_1.csv',
 'beta003_noNPI_vax_locom_detailed_2.csv',
 'beta003_noNPI_vax_locom_detailed_3.csv',
 'beta003_noNPI_vax_locom_detailed_4.csv',
 'beta006_noNPI_vax_locom_detailed_0.csv',
 'beta006_noNPI_vax_locom_detailed_1.csv',
 'beta006_noNPI_vax_locom_detailed_2.csv',
 'beta006_noNPI_vax_locom_detailed_3.csv',
 'beta006_noNPI_vax_locom_detailed_4.csv',
 'beta009_noNPI_vax_locom_detailed_0.csv',
 'beta009_noNPI_vax_locom_detailed_1.csv',
 'beta009_noNPI_vax_locom_detailed_2.csv',
 'beta009_noNPI_vax_locom_detailed_3.csv',
 'beta009_noNPI_vax_locom_detailed_4.csv']

In [13]:
df = pd.DataFrame()
for file in sim_names:
    tmpdf = pd.read_csv(file, header = 0, dtype = {'rep': int, 'tstep': int, 'p_vax': float, 
                                                                  'test_freq': int})
    part = int(file[-5])
    beta = int(file[6])
    tmpdf['partition'] = np.repeat(part, repeats = tmpdf.shape[0])
    tmpdf['beta'] = np.repeat(beta, repeats = tmpdf.shape[0])
    df = df.append(tmpdf, ignore_index = True)

In [14]:
# Linspace adds trailing decimal places in some cases
df['p_vax'] = df['p_vax'].round(decimals = 1)

In [15]:
df.shape

(847500, 13)

In [16]:
df.head()

,rep,p_vax,tstep,ext_inf_ct,test_freq,ia_nodes,is_nodes,test_pos_schd,test_pos_symp,q_schd,q_symp,partition,beta
0,1,0.0,0,44,0,[],[684],[],[],[],[],0,3
1,1,0.0,1,44,0,[],[684],[],[],[],[],0,3
2,1,0.0,2,44,0,[],[684],[],[],[],[],0,3
3,1,0.0,3,44,0,[403],"[684, 661]",[],[],[],[],0,3
4,1,0.0,4,44,0,[403],"[684, 661]",[],[684],[],[],0,3


In [17]:
# Renumber reps to remove duplicates
df['rep'] = df['rep'] + df['partition']*20
df.tail()

,rep,p_vax,tstep,ext_inf_ct,test_freq,ia_nodes,is_nodes,test_pos_schd,test_pos_symp,q_schd,q_symp,partition,beta
847495,100,0.8,108,10,28,[],[266],[74],[],"[218, 359, 489, 248]",[],4,9
847496,100,0.8,109,10,28,[],[266],"[500, 49]",[],"[74, 218, 359, 489, 248]",[],4,9
847497,100,0.8,110,10,28,[],[266],[],[266],"[49, 74, 218, 359, 489, 248]",[],4,9
847498,100,0.8,111,10,28,[],[266],[221],[],"[49, 74, 218, 359, 500, 248]",[266],4,9
847499,100,0.8,112,10,28,[],[266],[],[],"[49, 74, 218, 221, 359, 489, 500, 248]",[266],4,9


In [18]:
"""RESTRUCTURE DATA"""

df['ia_nodes'] = df['ia_nodes'].apply(eval)
df['is_nodes'] = df['is_nodes'].apply(eval)
df['all_inf'] = df['ia_nodes'] + df['is_nodes']
df['n_inf'] = df['all_inf'].apply(set).apply(len)
df['cum_list_inf'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_inf'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_inf'] = df['cum_list_inf'].apply(len)

df['test_pos_schd'] = df['test_pos_schd'].apply(eval)
df['test_pos_symp'] = df['test_pos_symp'].apply(eval)
df['all_test'] = df['test_pos_schd'] + df['test_pos_symp']
df['n_test_pos'] = df['all_test'].apply(set).apply(len)
df['cum_list_test'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_test'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_pos'] = df['cum_list_test'].apply(len)
        
df['q_schd'] = df['q_schd'].apply(eval)
df['q_symp'] = df['q_symp'].apply(eval)
df['all_quar'] = df['q_schd'] + df['q_symp']
df['n_quar'] = df['all_quar'].apply(set).apply(len)
df['cum_list_quar'] = df.groupby(['beta','rep','p_vax','test_freq'])['all_quar'].apply(lambda x: x.cumsum()).apply(set)
df['n_cum_quar'] = df['cum_list_quar'].apply(len)

In [ ]:
# Save out reformatted dataframe
filehandler = open(outfile + ".pkl","wb")
pickle.dump(df,filehandler)
filehandler.close()